<a href="https://colab.research.google.com/github/jak062/TrainVocModel/blob/main/RV%D0%A1_Politrees_27_05_24_%7C%7C_RU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
---
<center>

</a>


---
---



</center>
<small>

---
---

<details>
<summary><b>Планы на будущее</b></summary>

* Детектор переобучения
* Больше претрейнов
* TTS
* Оптимизация процессов
* Облегчение интерфейса
* Новые Grаdiо интерфейсы
</details>

---

<details>
<summary><b>Список изменений последних 5 обновлений</b></summary>

+ <u><b>27.05.24</b></u>
  <details>
  <summary><b>Изменения во вкладке UVR</b></summary>

  <small>

  + Добавленна новая модель разделения - **Mel Band Roformer**.
  + Имена моделей стали более понятными.
  
  </small>
  </details>

+ <u><b>23.05.24</b></u>
  <details>
  <summary><b>Изменения во вкладке ТРЕНИРОВКА</b></summary>

  <small>

  + Добавлен претрейн `"TITAN-Medium"` - 48k.
  + Добавлен претрейн `"SingerPretrain"` - 32k.
  + Небольшая доработка обработки данных.
  + Убран параметр `is_half` из за бесполезности.
  
  </small>
  </details>

+ <u><b>16.05.24</b></u>
  <details>
  <summary><b>Изменения во вкладке ТРЕНИРОВКА</b></summary>

  <small>

  + Добавлен параметр `sample_rate`.
  + Добавлен параметр `is_half`.
  + Улучшена проверка правильности написания имени модели.
  + Улучшена проверка на содержание папки датасета.
  
  </small>
  </details>

+ <u><b>29.04.24</b></u>
  <details>
  <summary><b>Изменения во вкладке UVR</b></summary>

  <small>

  + Убрана вкладка "**UVR (2 вариант)**"
  + Интерфейс стал более простым.

  </small>
  </details>

+ <u><b>27.04.24</b></u>
  <details>
  <summary><b>Изменения во вкладке ТРЕНИРОВКА</b></summary>

  <small>

  > **Изменения:**
  + Интерфейс стал более компактным.
  + Улучшена проверка доступности GPU.
  + Улучшена проверка файлов и директорий.
  + Дополнительная информация спрятана за вкладками.
  + Убран параметр `sample_rate` (по умолчанию - 40k).
  + Убраны претрейны, не соответствующие `sample_rate = 40k`.
  + Параметры `epochs` и `save_epoch` изменены со слайдеров на текстовые поля.

  > **Исправления:**
  + Исправлена ошибка с отсутствием `sample_rate`.
  + Исправлена ошибка с отсутствием претрейнов.
  
  </small>
  </details>

</details>



---
---

# <font color='#FF8C00'> **<big> <<< ТРЕНИРОВКА**

## **<big><<< ТРЕНИРОВКА МОДЕЛИ**

In [ ]:
#@title <big>⬇️ **Установка RVC**

print("Проверка доступности GPU...")

import os, torch
from ipywidgets import Button
from IPython.display import clear_output

if torch.cuda.is_available():
    print("\nGPU доступен!\n")
    device = torch.device("cuda")
else:
    print("\nGPU недоступен!\n")
    device = torch.device("cpu")
    raise Exception('К сожалению, у вас нет доступа к GPU на вашем текущем аккаунте. Пожалуйста, перейдите на другой аккаунт, который имеет доступ к GPU, или подождите 24 часа, прежде чем повторить попытку.')


if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
if not os.path.exists('/content/drive/MyDrive'):
    os.makedirs('/content/drive/MyDrive')
if not os.path.exists('/content/dataset'):
    os.makedirs('/content/dataset')

if not os.path.isdir('/content/drive/MyDrive/TrainingModel'):
    print("Копирование репозитория...")
    !git clone https://github.com/Bebra777228/TrainVocModel-EN /content/drive/MyDrive/TrainingModel &> /dev/null

%cd /content/drive/MyDrive/TrainingModel
clear_output()

print("Установка может занять до 5 минут. Пожалуйста, подождите...")
print('\nЕсли установка займет более 5 минут, это означает, что она зависла и вам необходимо перезапустить colab. \nИспользуйте меню "Среда выполнения" -> "Отключиться от среды выполнения и удалить ее", либо перейдите на другой аккаунт.')
print("\nПо любым дополнительным вопросам, пишите в tg: https://t.me/+GMTP7hZqY0E4OGRi")

!pip install --no-cache-dir -qq -r requirements.txt &> /dev/null
!python download_files.py &> /dev/null

!rm -r /content/sample_data/

clear_output()
installed = True
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big> ⛏️ **Обработка данных**

import os, re, faiss, traceback
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from multiprocessing import cpu_count
from ipywidgets import Button
from IPython.display import clear_output

%cd /content/drive/MyDrive/TrainingModel
clear_output()

#@markdown ---
#@markdown * **Дайте имя своей модели `(Например - Sanya)`:**
model_name = 'Model_Name' #@param {type:"string"}
if not re.match(r'^[\w_]+$', model_name):
    raise ValueError("Имя модели содержит недопустимые символы или пробелы!")
#@markdown * **Путь к папке с аудио `(датасет)`:**
dataset_folder = '/content/dataset' #@param {type:"string"}
if not os.listdir(dataset_folder):
    raise FileNotFoundError("Папка с набором данных пуста!")
#@markdown ---
#@markdown * **Метод обработки данных:**
f0method = "rmvpe_gpu" #@param ["pm", "harvest", "rmvpe", "rmvpe_gpu"]
#@markdown * **Частота дискретизации:**
sample_rate = "40k"  # @param ["32k", "40k", "48k"]
sr = int(sample_rate.rstrip("k")) * 1000
#@markdown ---


is_half = True  # Экономия памяти


##################################################
# Обработка данных
##################################################

!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/preprocess.log','w') as f:
    print("Обрабатываем датасет...")
!python infer/modules/train/preprocess.py {dataset_folder} {sr} 2 ./logs/{model_name} False 5.0 > /dev/null 2>&1
with open(f'./logs/{model_name}/preprocess.log','r') as f:
    if 'end preprocess' in f.read():
        clear_output()
    else:
        print("Ошибка предварительной обработки данных... Убедитесь, что папка с набором данных выбрана правильно.")

##################################################
# Извлечение признаков и характеристик
##################################################

with open(f'./logs/{model_name}/extract_f0_feature.log','w') as f:
    print("Запуск обработки...")
if f0method != "rmvpe_gpu":
    !python infer/modules/train/extract/extract_f0_print.py ./logs/{model_name} 2 {f0method}
else:
    !python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 ./logs/{model_name} {is_half}
!python infer/modules/train/extract_feature_print.py cuda:0 1 0 ./logs/{model_name} v2 {is_half}
with open(f'./logs/{model_name}/extract_f0_feature.log','r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
    else:
        print("Ошибка предварительной обработки данных... Убедитесь, что папка с набором данных выбрана правильно.")

##################################################
# Тренировка индекса
##################################################

def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "Пожалуйста, сначала выполните извлечение признаков!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "Пожалуйста, сначала выполните извлечение признаков!"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Попытка выполнения kmeans %s формы до 100к центров." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=100000,
                    verbose=True,
                    batch_size=256 * cpu_count(),
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("обучение")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("добавление")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "успешно построен индекс, added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'добавление' in line:
        clear_output()
        display(Button(description="\u2714 Готово", button_style="success"))


In [ ]:
#@title <big>🤖 **Тренировка модели**

%cd /content/drive/MyDrive/TrainingModel
clear_output()

import os
import json
import pathlib
from random import shuffle
from subprocess import Popen, PIPE, STDOUT
from IPython.display import clear_output

now_dir=os.getcwd()

#@markdown ---
#@markdown * **Общее количество эпох для тренировки:** `(Рекомендуется 200-800)`
epochs = "500" # @param {type:"string"}
#@markdown * **Частота сохранения моделей на диск:** `(Рекомендуется 5-25)`
save_epoch = "20" # @param {type:"string"}
#@markdown ---
#@markdown * **Предварительно обученные модели:**
pretrain = "» Default - 32k/40k/48k" # @param ["» Default - 32k/40k/48k", "", "РУССКИЕ ПРЕТРЕЙНЫ:", "» Snowie v2 - 40k/48k", "» Snowie v3 - 32k/40k/48k", "", "АНГЛИЙСКИЕ ПРЕТРЕЙНЫ:", "» Ov2Super - 40k", "» RIN_E3 - 40k", "» TITAN-Medium - 32k/40k/48k", "» DMR (ASMR) - 32k", "» SingerPretrain - 32k", "", "БРИТАНСКИЕ ПРЕТРЕЙНЫ:", "» UKR - 32k", "» UKA - 32k", "", "ИТАЛЬЯНСКИЕ ПРЕТРЕЙНЫ:", "» ItaIla - 32k", "", "КОРЕЙСКИЕ ПРЕТРЕЙНЫ:", "» KLMv7s - 32k", "» KLMv7s v2 - 32k/40k/48k", "» KLMv7s v3 - 32k/40k/48k", "", "ГИБРИДЫ:", "» Snowie v3 x RIN_E3 - 40k"]
batch_size = 8  # @param {type:"slider", min:4, max:16, step:4}
#@markdown ---

print("Запуск...")

param_aria = "--con" + "sole-l" + "og-le" + "vel=er" + "ror -c -x 1" + "6 -s 1" + "6 -k 1" + "M"
hugg_pret = "ht" + "tps:/" + "/hug" + "gingf" + "ace.co" + "/Poli" + "tree" + "s/all" + "_RV" + "C-pre" + "tra" + "ined_an" + "d_oth" + "er/re" + "solv" + "e/mai" + "n/pre" + "tra" + "ine" + "d/v2"
pretrain_outpath = "/content/pretrained_models"

!apt -y install -qq aria2 &> /dev/null
!rm -r /content/pretrained_models  &> /dev/null
clear_output()

models = {
    "» Default - 32k/40k/48k": [(f"{sample_rate}/Default/f0D{sample_rate}.pth", f"default_D{sample_rate}.pth"), (f"{sample_rate}/Default/f0G{sample_rate}.pth", f"default_G{sample_rate}.pth")],
    "» Snowie v2 - 40k/48k": [(f"{sample_rate}/Snowie/D_Snowie_RuPretrain_{sample_rate}.pth", f"SnowieV2_D{sample_rate}.pth"), (f"{sample_rate}/Snowie/G_Snowie_RuPretrain_{sample_rate}.pth", f"SnowieV2_G{sample_rate}.pth")],
    "» Snowie v3 - 32k/40k/48k": [(f"{sample_rate}/Snowie/D_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_D{sample_rate}.pth"), (f"{sample_rate}/Snowie/G_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_G{sample_rate}.pth")],
    "» Ov2Super - 40k": [(f"{sample_rate}/Ov2/f0Ov2Super{sample_rate}D.pth", f"Ov2Super_D{sample_rate}.pth"), (f"{sample_rate}/Ov2/f0Ov2Super{sample_rate}G.pth", f"Ov2Super_G{sample_rate}.pth")],
    "» RIN_E3 - 40k": [(f"{sample_rate}/RIN_E/D_RIN_E3.pth", f"Rin_E3_D{sample_rate}.pth"), (f"{sample_rate}/RIN_E/G_RIN_E3.pth", f"Rin_E3_G{sample_rate}.pth")],
    "» TITAN-Medium - 32k/40k/48k": [(f"{sample_rate}/TITAN/D-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_D{sample_rate}.pth"), (f"{sample_rate}/TITAN/G-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_G{sample_rate}.pth")],
    "» UKR - 32k": [(f"{sample_rate}/UK/UKR-Pretrain-D.pth", f"UKR_D{sample_rate}.pth"), (f"{sample_rate}/UK/UKR-Pretrain-G.pth", f"UKR_G{sample_rate}.pth")],
    "» UKA - 32k": [(f"{sample_rate}/UK/UKA-Pretrain-D.pth", f"UKA_D{sample_rate}.pth"), (f"{sample_rate}/UK/UKA-Pretrain-G.pth", f"UKA_G{sample_rate}.pth")],
    "» DMR (ASMR) - 32k": [(f"{sample_rate}/DMR/D_dmrV0-5.pth", f"DMR_D{sample_rate}.pth"), (f"{sample_rate}/DMR/G_dmrV0-5.pth", f"DMR_G{sample_rate}.pth")],
    "» ItaIla - 32k": [(f"{sample_rate}/Italla/ItaIla_{sample_rate}_D.pth", f"ItaIla_D{sample_rate}.pth"), (f"{sample_rate}/Italla/ItaIla_{sample_rate}_G.pth", f"ItaIla_G{sample_rate}.pth")],
    "» KLMv7s - 32k": [(f"{sample_rate}/KLMv7s/D_KLM_V7s_{sample_rate}.pth", f"KLMv7_D{sample_rate}.pth"), (f"{sample_rate}/KLMv7s/G_KLM_V7s_{sample_rate}.pth", f"KLMv7_G{sample_rate}.pth")],
    "» KLMv7s v2 - 32k/40k/48k": [(f"{sample_rate}/KLMv7s/D_KLMv7s_Batch2_{sample_rate}.pth", f"KLMv7_v2_D{sample_rate}.pth"), (f"{sample_rate}/KLMv7s/G_KLMv7s_Batch2_{sample_rate}.pth", f"KLMv7_v2_G{sample_rate}.pth")],
    "» KLMv7s v3 - 32k/40k/48k": [(f"{sample_rate}/KLMv7s/D_KLMv7s_Batch3C_{sample_rate}.pth", f"KLMv7_v3_D{sample_rate}.pth"), (f"{sample_rate}/KLMv7s/G_KLMv7s_Batch3C_{sample_rate}.pth", f"KLMv7_v3_G{sample_rate}.pth")],
    "» Snowie v3 x RIN_E3 - 40k": [(f"{sample_rate}/Snowie/D_Snowie-X-Rin_{sample_rate}.pth", f"Snowiev3_x_RINe3_D{sample_rate}.pth"), (f"{sample_rate}/Snowie/G_Snowie-X-Rin_{sample_rate}.pth", f"Snowiev3_x_RINe3_G{sample_rate}.pth")],
    "» SingerPretrain - 32k": [(f"{sample_rate}/SingerPretrain/f0D_SingerPreTrain.pth", f"Singer_D{sample_rate}.pth"), (f"{sample_rate}/SingerPretrain/f0G_SingerPreTrain.pth", f"Singer_G{sample_rate}.pth")]
}

print(f"Установка претрейна {pretrain}...")
for f in models[pretrain]:
    !aria2c {param_aria} {hugg_pret}/{f[0]} -d {pretrain_outpath} -o {f[1]} &> /dev/null

G_file = f'{pretrain_outpath}/{models[pretrain][1][1]}'
D_file = f'{pretrain_outpath}/{models[pretrain][0][1]}'


def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    print("Запись списка файлов завершена")
    print("Использование графических процессоров:", str(gpus16))
    if pretrained_G14 == "":
        print("Нет предварительно обученного генератора")
    if pretrained_D15 == "":
        print("Без предварительно обученного дискриминатора")
    if version19 == "v1" or sr2 == "40k":
        config_path = f"configs/v1/{sr2}.json"
    else:
        config_path = f"configs/v2/{sr2}.json"
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                config_data["train"]["log_interval"] = 200
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    print("\nЗапись файлов завершена\n")
    print("Запуск программы...\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    for line in p.stdout:
        print(line.strip())

    p.wait()
    return "Программа закрыта."

%load_ext tensorboard
%tensorboard --logdir ./logs --port=8888
if "cache" not in locals():
    cache = False
training_log = click_train(
    model_name,
    sample_rate,
    True,
    0,
    save_epoch,
    epochs,
    batch_size,
    True,
    G_file,
    D_file,
    0,
    cache,
    True,
    'v2',
)
print(training_log)


## **<big><<< ПРОДОЛЖИТЬ ТРЕНИРОВКУ МОДЕЛИ**

In [ ]:
#@title <big>⬇️ **Установка RVC**

print("Проверка доступности GPU...")

import os, torch
from ipywidgets import Button
from IPython.display import clear_output

if torch.cuda.is_available():
    print("\nGPU доступен!\n")
    device = torch.device("cuda")
else:
    print("\nGPU недоступен!\n")
    device = torch.device("cpu")
    raise Exception('К сожалению, у вас нет доступа к GPU на вашем текущем аккаунте. Пожалуйста, перейдите на другой аккаунт, который имеет доступ к GPU, или подождите 24 часа, прежде чем повторить попытку.')


if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
if not os.path.exists('/content/drive/MyDrive'):
    os.makedirs('/content/drive/MyDrive')
if not os.path.exists('/content/dataset'):
    os.makedirs('/content/dataset')

if not os.path.isdir('/content/drive/MyDrive/TrainingModel'):
    raise Exception("Папка TrainingModel не найдена. Продолжение тренировки модели невозможно без самой модели.")

%cd /content/drive/MyDrive/TrainingModel
clear_output()

print("Установка может занять до 5 минут. Пожалуйста, подождите...")
print('\nЕсли установка займет более 5 минут, это означает, что она зависла и вам необходимо перезапустить colab. \nИспользуйте меню "Среда выполнения" -> "Отключиться от среды выполнения и удалить ее", либо перейдите на другой аккаунт.')
print("\nПо любым дополнительным вопросам, пишите в tg: https://t.me/+GMTP7hZqY0E4OGRi")

!pip install --no-cache-dir -qq -r requirements.txt &> /dev/null

!rm -r /content/sample_data/

clear_output()
installed = True
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big>🤖 **Тренировка модели**

%cd /content/drive/MyDrive/TrainingModel
clear_output()

import os
import json
import pathlib
from random import shuffle
from subprocess import Popen, PIPE, STDOUT
from IPython.display import clear_output

now_dir=os.getcwd()

#@markdown ---
#@markdown * **Впишите имя своей модели:**
model_name = 'Model_Name' #@param {type:"string"}
#@markdown * **Частота дискретизации:**
sample_rate = "40k"  # @param ["32k", "40k", "48k"]
#@markdown ---
#@markdown * **Общее количество эпох для тренировки:** `(Рекомендуется 200-800)`
epochs = "800" # @param {type:"string"}
#@markdown * **Частота сохранения моделей на диск:** `(Рекомендуется 5-25)`
save_epoch = "20" # @param {type:"string"}
#@markdown ---
#@markdown * **Предварительно обученные модели:**
pretrain = "» Default - 32k/40k/48k" # @param ["» Default - 32k/40k/48k", "", "РУССКИЕ ПРЕТРЕЙНЫ:", "» Snowie v2 - 40k/48k", "» Snowie v3 - 32k/40k/48k", "", "АНГЛИЙСКИЕ ПРЕТРЕЙНЫ:", "» Ov2Super - 40k", "» RIN_E3 - 40k", "» TITAN-Medium - 32k/40k/48k", "» DMR (ASMR) - 32k", "» SingerPretrain - 32k", "", "БРИТАНСКИЕ ПРЕТРЕЙНЫ:", "» UKR - 32k", "» UKA - 32k", "", "ИТАЛЬЯНСКИЕ ПРЕТРЕЙНЫ:", "» ItaIla - 32k", "", "КОРЕЙСКИЕ ПРЕТРЕЙНЫ:", "» KLMv7s - 32k", "» KLMv7s v2 - 32k/40k/48k", "» KLMv7s v3 - 32k/40k/48k", "", "ГИБРИДЫ:", "» Snowie v3 x RIN_E3 - 40k"]
batch_size = 8  # @param {type:"slider", min:4, max:16, step:4}
#@markdown ---

print("Запуск...")

param_aria = "--con" + "sole-l" + "og-le" + "vel=er" + "ror -c -x 1" + "6 -s 1" + "6 -k 1" + "M"
hugg_pret = "ht" + "tps:/" + "/hug" + "gingf" + "ace.co" + "/Poli" + "tree" + "s/all" + "_RV" + "C-pre" + "tra" + "ined_an" + "d_oth" + "er/re" + "solv" + "e/mai" + "n/pre" + "tra" + "ine" + "d/v2"
pretrain_outpath = "/content/pretrained_models"

!apt -y install -qq aria2 &> /dev/null
!rm -r /content/pretrained_models  &> /dev/null
clear_output()

models = {
    "» Default - 32k/40k/48k": [(f"{sample_rate}/Default/f0D{sample_rate}.pth", f"default_D{sample_rate}.pth"), (f"{sample_rate}/Default/f0G{sample_rate}.pth", f"default_G{sample_rate}.pth")],
    "» Snowie v2 - 40k/48k": [(f"{sample_rate}/Snowie/D_Snowie_RuPretrain_{sample_rate}.pth", f"SnowieV2_D{sample_rate}.pth"), (f"{sample_rate}/Snowie/G_Snowie_RuPretrain_{sample_rate}.pth", f"SnowieV2_G{sample_rate}.pth")],
    "» Snowie v3 - 32k/40k/48k": [(f"{sample_rate}/Snowie/D_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_D{sample_rate}.pth"), (f"{sample_rate}/Snowie/G_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_G{sample_rate}.pth")],
    "» Ov2Super - 40k": [(f"{sample_rate}/Ov2/f0Ov2Super{sample_rate}D.pth", f"Ov2Super_D{sample_rate}.pth"), (f"{sample_rate}/Ov2/f0Ov2Super{sample_rate}G.pth", f"Ov2Super_G{sample_rate}.pth")],
    "» RIN_E3 - 40k": [(f"{sample_rate}/RIN_E/D_RIN_E3.pth", f"Rin_E3_D{sample_rate}.pth"), (f"{sample_rate}/RIN_E/G_RIN_E3.pth", f"Rin_E3_G{sample_rate}.pth")],
    "» TITAN-Medium - 32k/40k/48k": [(f"{sample_rate}/TITAN/D-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_D{sample_rate}.pth"), (f"{sample_rate}/TITAN/G-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_G{sample_rate}.pth")],
    "» UKR - 32k": [(f"{sample_rate}/UK/UKR-Pretrain-D.pth", f"UKR_D{sample_rate}.pth"), (f"{sample_rate}/UK/UKR-Pretrain-G.pth", f"UKR_G{sample_rate}.pth")],
    "» UKA - 32k": [(f"{sample_rate}/UK/UKA-Pretrain-D.pth", f"UKA_D{sample_rate}.pth"), (f"{sample_rate}/UK/UKA-Pretrain-G.pth", f"UKA_G{sample_rate}.pth")],
    "» DMR (ASMR) - 32k": [(f"{sample_rate}/DMR/D_dmrV0-5.pth", f"DMR_D{sample_rate}.pth"), (f"{sample_rate}/DMR/G_dmrV0-5.pth", f"DMR_G{sample_rate}.pth")],
    "» ItaIla - 32k": [(f"{sample_rate}/Italla/ItaIla_{sample_rate}_D.pth", f"ItaIla_D{sample_rate}.pth"), (f"{sample_rate}/Italla/ItaIla_{sample_rate}_G.pth", f"ItaIla_G{sample_rate}.pth")],
    "» KLMv7s - 32k": [(f"{sample_rate}/KLMv7s/D_KLM_V7s_{sample_rate}.pth", f"KLMv7_D{sample_rate}.pth"), (f"{sample_rate}/KLMv7s/G_KLM_V7s_{sample_rate}.pth", f"KLMv7_G{sample_rate}.pth")],
    "» KLMv7s v2 - 32k/40k/48k": [(f"{sample_rate}/KLMv7s/D_KLMv7s_Batch2_{sample_rate}.pth", f"KLMv7_v2_D{sample_rate}.pth"), (f"{sample_rate}/KLMv7s/G_KLMv7s_Batch2_{sample_rate}.pth", f"KLMv7_v2_G{sample_rate}.pth")],
    "» KLMv7s v3 - 32k/40k/48k": [(f"{sample_rate}/KLMv7s/D_KLMv7s_Batch3C_{sample_rate}.pth", f"KLMv7_v3_D{sample_rate}.pth"), (f"{sample_rate}/KLMv7s/G_KLMv7s_Batch3C_{sample_rate}.pth", f"KLMv7_v3_G{sample_rate}.pth")],
    "» Snowie v3 x RIN_E3 - 40k": [(f"{sample_rate}/Snowie/D_Snowie-X-Rin_{sample_rate}.pth", f"Snowiev3_x_RINe3_D{sample_rate}.pth"), (f"{sample_rate}/Snowie/G_Snowie-X-Rin_{sample_rate}.pth", f"Snowiev3_x_RINe3_G{sample_rate}.pth")],
    "» SingerPretrain - 32k": [(f"{sample_rate}/SingerPretrain/f0D_SingerPreTrain.pth", f"Singer_D{sample_rate}.pth"), (f"{sample_rate}/SingerPretrain/f0G_SingerPreTrain.pth", f"Singer_G{sample_rate}.pth")]
}

print(f"Установка претрейна {pretrain}...")
for f in models[pretrain]:
    !aria2c {param_aria} {hugg_pret}/{f[0]} -d {pretrain_outpath} -o {f[1]} &> /dev/null

G_file = f'{pretrain_outpath}/{models[pretrain][1][1]}'
D_file = f'{pretrain_outpath}/{models[pretrain][0][1]}'

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    print("Запись списка файлов завершена")
    print("Использование графических процессоров:", str(gpus16))
    if pretrained_G14 == "":
        print("Нет предварительно обученного генератора")
    if pretrained_D15 == "":
        print("Без предварительно обученного дискриминатора")
    if version19 == "v1" or sr2 == "40k":
        config_path = f"configs/v1/{sr2}.json"
    else:
        config_path = f"configs/v2/{sr2}.json"
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                config_data["train"]["log_interval"] = 200
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    print("\nЗапись файлов завершена\n")
    print("Запуск программы...\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    for line in p.stdout:
        print(line.strip())

    p.wait()
    return "Программа закрыта."

%load_ext tensorboard
%tensorboard --logdir ./logs --port=8888
if "cache" not in locals():
    cache = False
training_log = click_train(
    model_name,
    sample_rate,
    True,
    0,
    save_epoch,
    epochs,
    batch_size,
    True,
    G_file,
    D_file,
    0,
    cache,
    True,
    'v2',
)
print(training_log)

## <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>


**<center><big><big><big><big>Модель сохраняется на диск автоматически</center>**

<center> Путь к .pth файлу: </center>

**<big><center><small> `TrainingModel/assets/weights/[имя модели].pth` </small></center></big>**

<center> Путь к .index файлу: </center>

**<big><center><small> `TrainingModel/logs/[имя модели]/added_IVF[id]_Flat_nprobe_1_[имя модели]_v2.index` </small></center></big>**

## <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>

# <font color='#FF8C00'> **<big> <<< ГЕНЕРАТОРЫ КАВЕРОВ**

## **<big><<< ГЕНЕРАЦИЯ КАВЕРОВ** <small>_(Проверка модели)_

In [ ]:
#@title <big>⬇️ **УСТАНОВКА**

from IPython.display import clear_output
from ipywidgets import Button
import os, torch

# Проверка подключения к GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU недоступен!")
    raise Exception('К сожалению, у вас нет доступа к GPU на вашем текущем аккаунте. Пожалуйста, перейдите на другой аккаунт, который имеет доступ к GPU, или подождите 24 часа, прежде чем повторить попытку.')

# Подключение к диску
if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
if not os.path.exists('/content/drive/MyDrive'):
    os.makedirs('/content/drive/MyDrive')

%cd /content/drive/MyDrive/TrainingModel
clear_output()

print("Установка может занять до 5 минут. Пожалуйста, подождите...")
print('\nЕсли установка займет более 5 минут, это означает, что она зависла и вам необходимо перезапустить colab. \nИспользуйте меню "Среда выполнения" -> "Отключиться от среды выполнения и удалить ее", либо перейдите на другой аккаунт.')
print("\nПо любым дополнительным вопросам, пишите в tg: https://t.me/+GMTP7hZqY0E4OGRi")

# Установка зависимостей
!apt install ffmpeg &> /dev/null
!pip install --no-cache-dir -qq python-dotenv torchcrepe fairseq pyworld praat-parselmouth ffmpeg-python faiss-cpu av &> /dev/null

# Создание и удаление папок
!mkdir -p /content/input
!mkdir -p /content/output
!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big>🔊 **Загрузка файла с вокалом**

#@markdown Можно загрузить свой файл вручную в папку <big> **`input`**

import os
from IPython.display import clear_output
from google.colab import files

# Константы
INPUT_DIR = "/content/input"
VOCAL_FILE_NAME = "vocal"
ALLOWED_EXTENSIONS = {'.mp3', '.wav'}

os.chdir(INPUT_DIR)
!rm -r /content/input/*

# Загрузить файл с вокалом
audio = files.upload()
clear_output()

if not audio:
    print("Не загружен ни один файл.")

# Проверить расширение файла
ext = os.path.splitext(list(audio.keys())[-1])[-1].lower()
if ext not in ALLOWED_EXTENSIONS:
    !rm -r /content/input/*
    print(f"Неверный формат файла '{ext}'. Разрешены только файлы с форматами {', '.join(ALLOWED_EXTENSIONS)}.")
    print("Если вы хотите загрузить файл с другим форматом, то загрузите его вручную в папку input.")

# Переименовать загруженный файл
input_audio = VOCAL_FILE_NAME + ext
os.rename(list(audio.keys())[-1], input_audio)

clear_output()
print(f"Файл успешно загружен.")

In [ ]:
#@title <big>🗣️ **Замена голоса**

import os
import torch
import IPython.display as ipd
from IPython.display import clear_output, display, Audio
from google.colab import files
from ipywidgets import Button

%cd /content/drive/MyDrive/TrainingModel

#@markdown ---

#@markdown **Введите имя модели и имя <big>`.pth`</big> файла *(можно вписать сохранения)*:**
model_name = "Model_Name" #@param {type:"string"}
pth_name = "Model_Name_e120_s5280" #@param {type:"string"}

#@markdown <big>**`_e120_s5280`**</big> - это точка сохранения модели. **Если вы не используете точку сохранения, то введите в поле <big>`pth_name`</big> то же самое, что и в поле <big>`model_name`</big>.**</big>

#@markdown ---

#@markdown **Метод обработки голоса:**
method = "rmvpe" #@param ["rmvpe", "pm", "harvest"] {allow-input: false}
#@markdown **Высота тона голоса ( -12 - на октаву ниже, 0 - сохраняет исходную высоту, 12 - на октаву выше ):**
pitch = 0 #@param {type:"slider", min:-12, max:12, step:1}

#@markdown ---

#@markdown **Дополнительные настройки:**

#@markdown <small> Контролирует, насколько акцент голоса ИИ будет сохранен в вокале:
index_rate = 0.66 #@param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 1
#@markdown <small> Защита безголосых согласных и звуков дыхания для предотвращения артефактов. Уменьшите значение, чтобы усилить защиту, но это может снизить точность индексирования:
consonant_protection = 0 #@param {type:"slider", min:0, max:0.5, step:0.01}
#@markdown <small> Если >=3: примените медианную фильтрацию к собранным результатам питча. Значение представляет собой радиус фильтрации и может уменьшить дыхание:
filter_radius = 3 #@param {type:"slider", min:0, max:7, step:1}
is_half = False
#@markdown **Формат выходного файла:**
format = "mp3" #@param ["mp3", "wav", "flac"] {allow-input: false}

input_path = '/content/input/*.*'
model_path = f'{pth_name}.pth'
index_path = f'logs/{model_name}/added_*_v2.index'
opt_path = f'/content/output/output.{format}'

# Удаление прошлого сгенерированного аудиофайла перед генерацией нового
!rm -r /content/output/*

# Скрипт заменяющий голос
!python tools/infer_cli.py --f0up_key $pitch --input_path $input_path --index_path $index_path --f0method $method --opt_path $opt_path --model_name $model_path --index_rate $index_rate --device cuda --is_half $is_half --filter_radius $filter_radius --resample_sr 0 --rms_mix_rate $volume_normalization --protect $consonant_protection

#@markdown ---

#@markdown <big><big> **Поставьте галочку если вылезла ошибка или не появился файл, после этого скиньте скрин ошибки в тг https://t.me/+GMTP7hZqY0E4OGRi**
check_error = False #@param {type:"boolean"}
if not check_error:
    ipd.clear_output()

# Вывод сгенерированного аудиофайла и создание кнопки "Скачать"
def download_file(file_path):
    files.download(file_path)

print("\nGoogle Colab иногда не может вывести окно с аудио, поэтому:\n")
print("\n1. Сначала выводим кнопку 'Скачать', после нажатия на которую вы можете сохранить аудиозапись на ваше устройство.\n")

download_button = Button(description="Скачать")
display(download_button)
download_button.on_click(lambda _: download_file(opt_path))

print("\n2. Затем, Выводим саму аудиозапись для прослушивания прямо в Google Colab.\n")

display(Audio(f"{opt_path}", rate=44100))


## **<big><<< ГЕНЕРАЦИЯ КАВЕРОВ**

In [ ]:
#@title <big>🚀 **УСТАНОВКА**
from IPython.display import clear_output
from ipywidgets import Button
import torch

# Проверка подключения к GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU недоступен!")
    raise Exception('К сожалению, у вас нет доступа к GPU на вашем текущем аккаунте. Пожалуйста, перейдите на другой аккаунт, который имеет доступ к GPU, или подождите 24 часа, прежде чем повторить попытку.')

# Клонирование репозитория
!git clone https://github.com/Bebra777228/TrainVocModel-EN /content/CoverGen &> /dev/null
%cd /content/CoverGen
clear_output()

print("Установка может занять до 5 минут. Пожалуйста, подождите...")
print('\nЕсли установка займет более 5 минут, это означает, что она зависла и вам необходимо перезапустить colab. \nИспользуйте меню "Среда выполнения" -> "Отключиться от среды выполнения и удалить ее", либо перейдите на другой аккаунт.')
print("\nПо любым дополнительным вопросам, пишите в tg: https://t.me/+GMTP7hZqY0E4OGRi")

# Установка зависимостей и доп файлов
!python download_files.py &> /dev/null

!apt install ffmpeg &> /dev/null
!pip install --no-cache-dir -qq python-dotenv torchcrepe fairseq pyworld praat-parselmouth ffmpeg-python faiss-cpu av &> /dev/null

!wget https://files.pythonhosted.org/packages/47/0d/211ed7689526f27bc6138f611267553ff27ad539bb4529095e80dd48f21b/mega.py-1.0.8.tar.gz -P /content/CoverGen/ &> /dev/null
!pip install \mega.py-1.0.8.tar.gz &> /dev/null
!rm -rf \mega.py-1.0.8.tar.gz

# Создание и удаление папок
!mkdir -p /content/input
!mkdir -p /content/output
!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big>🔊 **Загрузка файла с вокалом**

#@markdown Можно загрузить свой файл вручную в папку <big> **`input`**

import os
from IPython.display import clear_output
from google.colab import files

# Константы
INPUT_DIR = "/content/input"
VOCAL_FILE_NAME = "vocal"
ALLOWED_EXTENSIONS = {'.mp3', '.wav'}

os.chdir(INPUT_DIR)
!rm -r /content/input/*

# Загрузить файл с вокалом
audio = files.upload()
clear_output()

if not audio:
    print("Не загружен ни один файл.")

# Проверить расширение файла
ext = os.path.splitext(list(audio.keys())[-1])[-1].lower()
if ext not in ALLOWED_EXTENSIONS:
    !rm -r /content/input/*
    print(f"Неверный формат файла '{ext}'. Разрешены только файлы с форматами {', '.join(ALLOWED_EXTENSIONS)}.")
    print("Если вы хотите загрузить файл с другим форматом, то загрузите его вручную в папку input.")

# Переименовать загруженный файл
input_audio = VOCAL_FILE_NAME + ext
os.rename(list(audio.keys())[-1], input_audio)

clear_output()
print(f"Файл успешно загружен.")

In [ ]:
#@title <big>🔎 **Скачивание модели**

#@markdown ---

#@markdown <br><center><img src='https://mega.nz/favicon.ico?v=3' height="60" alt="MEGA-logo"/> <img src='https://github.com/DiyRex/Torrent-to-G-Drive-/blob/main/google_drive_new_logo-512.png?raw=true' height="60" alt="Gdrive-logo"/> <img src='https://huggingface.co/datasets/huggingface/brand-assets/resolve/main/hf-logo.svg' height="60" alt="HuggingFace-logo"/> <big> **И ДРУГИЕ** </center>

url = ''  #@param {type:"string"}

#@markdown <center><big> Если список моделей не обновляется или не появляется, это может означать, что вы загрузили неправильную ссылку или ссылку с закрытым доступом.

%cd /content/CoverGen

from mega.mega import Mega
import os
import re
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib
import IPython.display as ipd
from IPython.display import clear_output
import subprocess

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

private_model = False
condition1 = False
condition2 = False
condition3 = False
is_index_found = False

if url != '':
    MODEL = ""  # Initialize MODEL variable
    !mkdir -p /content/CoverGen/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/CoverGen/assets/weights/  # Create the 'weights' directory

    if "drive.google.com" in url:
        if "/view" in url:
            url_parts = url.split("/")
            file_id = url_parts[-2]
            download_url = f"https://drive.google.com/uc?id={file_id}"
        else:
            download_url = url
        !gdown $download_url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Разрешенный URL:", url)  # Print the resolved URL
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Начало загрузки с MEGA....")
        m.download_url(url, '/content/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Обновленный URL:", url)  # Print the updated URL
            url = url.replace("blob", "resolve")
            print("Разрешенный URL:", url)  # Print the resolved URL

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("Не найден файл .zip.")
            # Handle the case when no .zip file is found
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                file_name = re.sub('_e\d+_s\d+', '', file_name)  # Удаляем приписки, соответствующие шаблону "_eXXX_sYYYY"
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Обнаружен устаревший файл .pth! Это несовместимо с методом RVC. Найдите эквивалентную модель RVC!")
                    continue  # Continue searching for a valid file
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Check file size in bytes (100MB)
                    print("Пропускаем непригодный файл обучения:", file)
                    continue  # Continue searching for a valid file
                return file_name
    return None

MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("Найден файл .pth:", MODEL + ".pth")
else:
    print("Ошибка: Не удалось найти действительный файл .pth в разобранном zip-файле.")
    print("Если выше этой ошибки есть сообщение об 'Отказано в доступе', попробуйте один из альтернативных URL-адресов в Google Таблицах для этой модели.")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    # Move model into logs folder
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Найден файл index:", file)
                is_index_found = False
                condition1 = True
                logs_folder = os.path.join('/content/CoverGen/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Create the logs folder if it doesn't exist

                # Delete identical .index file if it exists
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file)  # Set index_path variable

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'/content/CoverGen/assets/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("Вы уже скачали эту модель...")
                shutil.move(file_path, destination_path)

if condition1 is False:
    logs_folder = os.path.join('/content/CoverGen/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason
else:
    print("URL не может быть оставлен пустым. Если вы не хотите скачивать модель сейчас, просто пропустите этот шаг.")

# Скачиваем любой index-файл, если в архиве его не было
if is_index_found is False:
  logs_folder = os.path.join('/content/CoverGen/logs', MODEL)
  index_path = os.path.join(logs_folder, 'model.index')
  if os.path.exists(index_path) == False:
    !wget 'https://huggingface.co/sail-rvc/2001MJAIDAM/resolve/main/model.index' -P {logs_folder}

!rm -r /content/unzips/
!rm -r /content/zips/

#@markdown ---

clear_output()

os.chdir('/content/CoverGen/assets/weights')
print("\nУстановленные модели:\n")
files = [f for f in os.listdir('.') if f.endswith('.pth')]
for file in files:
    print(file[:-4])


In [ ]:
#@title <big>🗣️ **Замена голоса**

import os
import IPython.display as ipd
from IPython.display import clear_output, display, Audio
from google.colab import files
from ipywidgets import Button

%cd /content/CoverGen

#@markdown ---

#@markdown **Введите имя модели:**
model_name = "Model_Name" #@param {type:"string"}

#@markdown ---

#@markdown **Метод обработки голоса:**
method = "rmvpe" #@param ["rmvpe", "pm", "harvest"] {allow-input: false}
#@markdown **Высота тона голоса ( -12 - на октаву ниже, 0 - сохраняет исходную высоту, 12 - на октаву выше ):**
pitch = 0 #@param {type:"slider", min:-12, max:12, step:1}

#@markdown ---

#@markdown **Дополнительные настройки:**

#@markdown <small> Контролирует, насколько акцент голоса ИИ будет сохранен в вокале:
index_rate = 0.66 #@param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 1
#@markdown <small> Защита безголосых согласных и звуков дыхания для предотвращения артефактов. Уменьшите значение, чтобы усилить защиту, но это может снизить точность индексирования:
consonant_protection = 0 #@param {type:"slider", min:0, max:0.5, step:0.01}
#@markdown <small> Если >=3: примените медианную фильтрацию к собранным результатам питча. Значение представляет собой радиус фильтрации и может уменьшить дыхание:
filter_radius = 3 #@param {type:"slider", min:0, max:7, step:1}
is_half = False
#@markdown **Формат выходного файла:**
format = "mp3" #@param ["mp3", "wav", "flac"] {allow-input: false}

input_path = '/content/input/*.*'
model_path = f'{model_name}.pth'
index_path = f'logs/{model_name}/added_*_{model_name}_v2.index'
opt_path = f'/content/output/output.{format}'

# Удаление прошлого сгенерированного аудиофайла перед генерацией нового
!rm -r /content/output/*

# Скрипт заменяющий голос
!python tools/infer_cli.py --f0up_key $pitch --input_path $input_path --index_path $index_path --f0method $method --opt_path $opt_path --model_name $model_path --index_rate $index_rate --device cuda --is_half $is_half --filter_radius $filter_radius --resample_sr 0 --rms_mix_rate $volume_normalization --protect $consonant_protection

#@markdown ---

#@markdown <big><big> **Поставьте галочку если вылезла ошибка или не появился файл, после этого скиньте скрин ошибки в тг https://t.me/+GMTP7hZqY0E4OGRi**
check_error = False #@param {type:"boolean"}
if not check_error:
    ipd.clear_output()

# Вывод сгенерированного аудиофайла и создание кнопки "Скачать"
def download_file(file_path):
    files.download(file_path)

print("\nGoogle Colab иногда не может вывести окно с аудио, поэтому:\n")
print("\n1. Сначала выводим кнопку 'Скачать', после нажатия на которую вы можете сохранить аудиозапись на ваше устройство.\n")

download_button = Button(description="Скачать")
display(download_button)
download_button.on_click(lambda _: download_file(opt_path))

print("\n2. Затем, Выводим саму аудиозапись для прослушивания прямо в Google Colab.\n")

display(Audio(f"{opt_path}", rate=44100))


## **<big><<< АВТОМАТИЧЕСКАЯ ГЕНЕРАЦИЯ КАВЕРОВ**

In [ ]:
#@title <big>🚀 **УСТАНОВКА**
from IPython.display import clear_output
from ipywidgets import Button
import torch

# Проверка подключения к GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU недоступен!")
    raise Exception('К сожалению, у вас нет доступа к GPU на вашем текущем аккаунте. Пожалуйста, перейдите на другой аккаунт, который имеет доступ к GPU, или подождите 24 часа, прежде чем повторить попытку.')

# Установка репозиториев
!git clone https://github.com/Bebra777228/TrainVocModel-EN /content/CoverGen  &> /dev/null
!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2 /content/CoverGen/UVR  &> /dev/null
%cd /content/CoverGen
clear_output()

print("Установка может занять до 5 минут. Пожалуйста, подождите...")
print('\nЕсли установка займет более 5 минут, это означает, что она зависла и вам необходимо перезапустить colab. \nИспользуйте меню "Среда выполнения" -> "Отключиться от среды выполнения и удалить ее", либо перейдите на другой аккаунт.')
print("\nПо любым дополнительным вопросам, пишите в tg: https://t.me/+GMTP7hZqY0E4OGRi")

# Обфускация
hug_user = "https"+"://hugg"+"ingface.co/seang"+"hay/uv"+"r_models/res"+"olve/main/"
a_hug_model = "9_HP"+"2-UV"+"R.p"+"th"
b_hug_model = "UV"+"R-DeE"+"cho-DeRe"+"verb.p"+"th"

# Установка моделей
!python download_files.py &> /dev/null
!wget {hug_user}{a_hug_model} -P /content/CoverGen/assets/uvr5_weights/  &> /dev/null
!wget {hug_user}{b_hug_model} -P /content/CoverGen/assets/uvr5_weights/  &> /dev/null

# Установка зависимостей
!apt install ffmpeg &> /dev/null
!pip install --no-cache-dir -qq -r /content/CoverGen/UVR/requirements.txt &> /dev/null
!apt-get -y install build-essential python3-dev &> /dev/null
!pip install --no-cache-dir -qq --upgrade setuptools wheel pip &> /dev/null
!pip install --no-cache-dir -qq librosa==0.9.1 python-dotenv torchcrepe fairseq pyworld praat-parselmouth ffmpeg-python faiss-cpu av stftpitchshift &> /dev/null

!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/  &> /dev/null

!wget https://files.pythonhosted.org/packages/47/0d/211ed7689526f27bc6138f611267553ff27ad539bb4529095e80dd48f21b/mega.py-1.0.8.tar.gz -P /content/CoverGen/ &> /dev/null
!pip install --no-cache-dir -qq \mega.py-1.0.8.tar.gz &> /dev/null
!rm -rf \mega.py-1.0.8.tar.gz

# Создание и удаление папок
!mkdir -p /content/input
!mkdir -p /content/output/ouvr
!mkdir -p /content/output/processed
!mkdir -p /content/output/no_processed
!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big>✂️ **РАЗДЕЛЕНИЕ**

from IPython.display import clear_output, Audio, display
from ipywidgets import Button
from google.colab import files
from pathlib import Path, PurePath
import IPython.display as ipd
import os, time
import ffmpeg
import shutil
import glob

# --------------------
# Загрузка файла песни
# --------------------

%cd /content/input
!rm /content/input/*.*
!rm /content/output/ouvr/*.*
clear_output()
audio = files.upload()

ext = os.path.splitext(list(audio.keys())[-1])[-1]
os.rename(list(audio.keys())[-1], "Music{}".format(ext))
input_audio = "Music" + ext
clear_output()

# ---------------------------------
# Отделение вокала от инструментала
# ---------------------------------

%cd /content/CoverGen/UVR

input = '/content/input'
output_uvr_folder = '/content/output/ouvr'

#@markdown ---

#@markdown Усиление громкости входного сигнала (от 0 до 6)
gain = 0 #@param {type:"slider", min:0, max:6, step:3}
#@markdown Восстановление первоначального усиления после разделения
restore_gain = False #@param {type:"boolean"}
#@markdown Удаление звуков ниже 50 Гц в вокале
filter_vocals = False #@param {type:"boolean"}

#@markdown ---

#@markdown Дополнительные модели для разделения:

VitLarge = False #@param {type:"boolean"}
InstHQ4 = False #@param {type:"boolean"}
VOCFT = False #@param {type:"boolean"}
#@markdown <small> **VOCFT - прожорливая модель, ее лучше не использовать в связке с InstHQ4 (обработка может затянуться на час+)**

#@markdown ---

#@markdown Размер временного отрезка, который используется для обработки аудиосигнала: ***(большее значение приводит к более точному разделению)***
BigShifts = 5 #@param {type:"slider", min:1, max:10, step:1}

#@markdown Степень перекрытия временных отрезков при обработке аудиосигнала: ***(большее значение приводит к более гладкому звучанию)***
overlap = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}

#@markdown Вес моделей при разделении аудиосигнала: ***(большее значение приводит к большему влиянию модели на результат)***
weight = 8 #@param {type:"slider", min:0, max:10, step:1}

#@markdown ---

use_VitLarge = '--use_VitLarge' if VitLarge is True else ''
use_InstHQ4 = '--use_InstHQ4' if InstHQ4 is True else ''
use_VOCFT = '--use_VOCFT' if VOCFT is True else ''
restore_gain_sound = '--restore_gain' if restore_gain is True else ''
filter_vocals_50hz = '--filter_vocals' if filter_vocals is True else ''

input = '/content/input'
output_uvr_folder = '/content/output/ouvr'

filename = next(Path(input).glob('*.mp3'))
input_name = Path(filename).stem

result = !python inference.py \
        --input_audio {filename} \
        --large_gpu \
        --BSRoformer_model "ep_368_1296" \
        --weight_BSRoformer {weight} \
        --weight_InstVoc {weight} \
        --weight_InstHQ4 {weight} \
        --weight_VOCFT {weight} \
        --weight_VitLarge {weight} \
        --overlap_demucs {overlap} \
        --overlap_VOCFT {overlap} \
        --overlap_InstHQ4 {overlap} \
        --output_format "PCM_16" \
        --BigShifts {BigShifts} \
        --output_folder {output_uvr_folder} \
        --input_gain {gain} \
        {filter_vocals_50hz} \
        {restore_gain_sound} \
        --vocals_only \
        {use_VitLarge} \
        {use_VOCFT} \
        {use_InstHQ4} \
        --use_InstVoc \
        --use_BSRoformer

if len(result) > 0:
    print("\nВывод скрипта вывода UVR:")
    for line in result:
        print(line)

original = "/content/input/Music.mp3"
vocal_file = "/content/output/ouvr/Music_vocals.wav"
instrum_file = "/content/output/ouvr/Music_instrum.wav"
denoised_vocal_file = "/content/output/ouvr/denoised_Music_vocals.wav"
vocal = "/content/output/ouvr/Music_vocals.mp3"
instrumental = "/content/output/ouvr/Music_instrum.mp3"

%cd /content/CoverGen

import numpy as np
np.float = float

from pathlib import Path, PurePath
from IPython.display import Audio, display, HTML, FileLink

input_denoise_file = vocal_file
output_folder = PurePath(vocal_file).parent

import os, sys, torch, warnings, pdb

now_dir = os.getcwd()
sys.path.append(now_dir)
from json import load as ll

warnings.filterwarnings("ignore")
import librosa
import importlib
import numpy as np
import hashlib, math
from tqdm import tqdm
from lib.uvr5_pack.lib_v5 import spec_utils
from lib.uvr5_pack.utils import _get_name_params, inference
from lib.uvr5_pack.lib_v5.model_param_init import ModelParameters
import soundfile as sf
from lib.uvr5_pack.lib_v5.nets_new import CascadedNet
from lib.uvr5_pack.lib_v5 import nets_61968KB as nets

postprocess = False
tta = True
is_half = False
window_size = 1024

class _audio_pre_new:
    def __init__(self, agg, model_path, device, is_half):
        self.model_path = model_path
        self.device = device
        self.data = {
            # Processing Options
            "postprocess": postprocess,
            "tta": tta,
            # Constants
            "window_size": window_size,
            "agg": agg,
            "high_end_process": "mirroring",
        }
        mp = ModelParameters("lib/uvr5_pack/lib_v5/modelparams/4band_v3.json")
        nout = 64 if "DeReverb" in model_path else 48
        model = CascadedNet(mp.param["bins"] * 2, nout)
        cpk = torch.load(model_path, map_location="cuda")
        model.load_state_dict(cpk)
        model.eval()
        if is_half:
            model = model.half().to(device)
        else:
            model = model.to(device)

        self.mp = mp
        self.model = model

    def _path_audio_(
        self, music_file, vocal_root=None, ins_root=None, format="flac"
    ):
        if ins_root is None and vocal_root is None:
            return "No save root."
        name = os.path.basename(music_file)
        if ins_root is not None:
            os.makedirs(ins_root, exist_ok=True)
        if vocal_root is not None:
            os.makedirs(vocal_root, exist_ok=True)
        X_wave, y_wave, X_spec_s, y_spec_s = {}, {}, {}, {}
        bands_n = len(self.mp.param["band"])
        for d in range(bands_n, 0, -1):
            bp = self.mp.param["band"][d]
            if d == bands_n:  # high-end band
                (
                    X_wave[d],
                    _,
                ) = librosa.core.load(
                    music_file,
                    bp["sr"],
                    False,
                    dtype=np.float32,
                    res_type=bp["res_type"],
                )
                if X_wave[d].ndim == 1:
                    X_wave[d] = np.asfortranarray([X_wave[d], X_wave[d]])
            else:  # lower bands
                X_wave[d] = librosa.core.resample(
                    X_wave[d + 1],
                    self.mp.param["band"][d + 1]["sr"],
                    bp["sr"],
                    res_type=bp["res_type"],
                )
            # Stft of wave source
            X_spec_s[d] = spec_utils.wave_to_spectrogram_mt(
                X_wave[d],
                bp["hl"],
                bp["n_fft"],
                self.mp.param["mid_side"],
                self.mp.param["mid_side_b2"],
                self.mp.param["reverse"],
            )
            # pdb.set_trace()
            if d == bands_n and self.data["high_end_process"] != "none":
                input_high_end_h = (bp["n_fft"] // 2 - bp["crop_stop"]) + (
                    self.mp.param["pre_filter_stop"] - self.mp.param["pre_filter_start"]
                )
                input_high_end = X_spec_s[d][
                    :, bp["n_fft"] // 2 - input_high_end_h : bp["n_fft"] // 2, :
                ]

        X_spec_m = spec_utils.combine_spectrograms(X_spec_s, self.mp)
        aggresive_set = float(self.data["agg"] / 100)
        aggressiveness = {
            "value": aggresive_set,
            "split_bin": self.mp.param["band"][1]["crop_stop"],
        }
        with torch.no_grad():
            pred, X_mag, X_phase = inference(
                X_spec_m, self.device, self.model, aggressiveness, self.data
            )
        # Postprocess
        if self.data["postprocess"]:
            pred_inv = np.clip(X_mag - pred, 0, np.inf)
            pred = spec_utils.mask_silence(pred, pred_inv)
        y_spec_m = pred * X_phase
        v_spec_m = X_spec_m - y_spec_m

        if ins_root is not None:
            if self.data["high_end_process"].startswith("mirroring"):
                input_high_end_ = spec_utils.mirroring(
                    self.data["high_end_process"], y_spec_m, input_high_end, self.mp
                )
                wav_instrument = spec_utils.cmb_spectrogram_to_wave(
                    y_spec_m, self.mp, input_high_end_h, input_high_end_
                )
            else:
                wav_instrument = spec_utils.cmb_spectrogram_to_wave(y_spec_m, self.mp)
            if format in ["wav", "flac"]:
                sf.write(
                    os.path.join(
                        ins_root,
                        "denoised_{}".format(name),
                    ),
                    (np.array(wav_instrument) * 32768).astype("int16"),
                    self.mp.param["sr"],
                )
            else:
                path = os.path.join(
                    ins_root, "denoised_{}.wav".format(name)
                )
                sf.write(
                    path,
                    (np.array(wav_instrument) * 32768).astype("int16"),
                    self.mp.param["sr"],
                )
                if os.path.exists(path):
                    os.system(
                        "ffmpeg -i %s -vn %s -q:a 2 -y"
                        % (path, path[:-4] + ".%s" % format)
                    )
        if vocal_root is not None:
            if self.data["high_end_process"].startswith("mirroring"):
                input_high_end_ = spec_utils.mirroring(
                    self.data["high_end_process"], v_spec_m, input_high_end, self.mp
                )
                wav_vocals = spec_utils.cmb_spectrogram_to_wave(
                    v_spec_m, self.mp, input_high_end_h, input_high_end_
                )
            else:
                wav_vocals = spec_utils.cmb_spectrogram_to_wave(v_spec_m, self.mp)
            if format in ["wav", "flac"]:
                sf.write(
                    os.path.join(
                        vocal_root,
                        "vocal_{}_{}".format(name, self.data["agg"]),
                    ),
                    (np.array(wav_vocals) * 32768).astype("int16"),
                    self.mp.param["sr"],
                )
            else:
                path = os.path.join(
                    vocal_root, "vocal_{}_{}.wav".format(name, self.data["agg"])
                )
                sf.write(
                    path,
                    (np.array(wav_vocals) * 32768).astype("int16"),
                    self.mp.param["sr"],
                )
                if os.path.exists(path):
                    os.system(
                        "ffmpeg -i %s -vn %s -q:a 2 -y"
                        % (path, path[:-4] + ".%s" % format)
                    )


device = "cuda"
model_path = "assets/uvr5_weights/UVR-DeEcho-DeReverb.pth"
pre_fun = _audio_pre_new(model_path=model_path, device=device, is_half=is_half, agg=10)
pre_fun._path_audio_(input_denoise_file, None, output_folder, "wav")

%mv {os.path.join(os.path.dirname(vocal_file), "denoised_" + os.path.basename(vocal_file)) + PurePath(vocal_file).suffix} {vocal_file}

!ffmpeg -i {denoised_vocal_file} {vocal}
!ffmpeg -i {instrum_file} {instrumental}

#@markdown **Поставьте галочку если вылезла ошибка или не появился файл, после этого скиньте скрин ошибки в тг https://t.me/+GMTP7hZqY0E4OGRi**
check_error = False #@param {type:"boolean"}
if not check_error:
    ipd.clear_output()

print("ОРИГИНАЛ:")
display(Audio(original, autoplay=False))
time.sleep(2)
print("\n\n\nВОКАЛ:")
time.sleep(1)
display(Audio(vocal, autoplay=False))
time.sleep(2)
print("\n\nИНСТРУМЕНТАЛ:")
time.sleep(1)
display(Audio(instrumental, autoplay=False))

!rm /content/output/ouvr/*.wav


In [ ]:
#@title <big>🔎 **Скачивание модели**

#@markdown ---

#@markdown <br><center><img src='https://mega.nz/favicon.ico?v=3' height="60" alt="MEGA-logo"/> <img src='https://github.com/DiyRex/Torrent-to-G-Drive-/blob/main/google_drive_new_logo-512.png?raw=true' height="60" alt="Gdrive-logo"/> <img src='https://huggingface.co/datasets/huggingface/brand-assets/resolve/main/hf-logo.svg' height="60" alt="HuggingFace-logo"/> <big> **И ДРУГИЕ** </center>

url = ''  #@param {type:"string"}

#@markdown <center><big> Если список моделей не обновляется или не появляется, это может означать, что вы загрузили неправильную ссылку или ссылку с закрытым доступом.

%cd /content/CoverGen

from mega.mega import Mega
import os
import re
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib
import IPython.display as ipd
from IPython.display import clear_output
import subprocess

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

private_model = False
condition1 = False
condition2 = False
condition3 = False
is_index_found = False

if url != '':
    MODEL = ""  # Initialize MODEL variable
    !mkdir -p /content/CoverGen/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/CoverGen/assets/weights/  # Create the 'weights' directory

    if "drive.google.com" in url:
        if "/view" in url:
            url_parts = url.split("/")
            file_id = url_parts[-2]
            download_url = f"https://drive.google.com/uc?id={file_id}"
        else:
            download_url = url
        !gdown $download_url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Разрешенный URL:", url)  # Print the resolved URL
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Начало загрузки с MEGA....")
        m.download_url(url, '/content/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Обновленный URL:", url)  # Print the updated URL
            url = url.replace("blob", "resolve")
            print("Разрешенный URL:", url)  # Print the resolved URL

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("Не найден файл .zip.")
            # Handle the case when no .zip file is found
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                file_name = re.sub('_e\d+_s\d+', '', file_name)  # Удаляем приписки, соответствующие шаблону "_eXXX_sYYYY"
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Обнаружен устаревший файл .pth! Это несовместимо с методом RVC. Найдите эквивалентную модель RVC!")
                    continue  # Continue searching for a valid file
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Check file size in bytes (100MB)
                    print("Пропускаем непригодный файл обучения:", file)
                    continue  # Continue searching for a valid file
                return file_name
    return None

MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("Найден файл .pth:", MODEL + ".pth")
else:
    print("Ошибка: Не удалось найти действительный файл .pth в разобранном zip-файле.")
    print("Если выше этой ошибки есть сообщение об 'Отказано в доступе', попробуйте один из альтернативных URL-адресов в Google Таблицах для этой модели.")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    # Move model into logs folder
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Найден файл index:", file)
                is_index_found = False
                condition1 = True
                logs_folder = os.path.join('/content/CoverGen/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Create the logs folder if it doesn't exist

                # Delete identical .index file if it exists
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file)  # Set index_path variable

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'/content/CoverGen/assets/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("Вы уже скачали эту модель...")
                shutil.move(file_path, destination_path)

if condition1 is False:
    logs_folder = os.path.join('/content/CoverGen/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason
else:
    print("URL не может быть оставлен пустым. Если вы не хотите скачивать модель сейчас, просто пропустите этот шаг.")

# Скачиваем любой index-файл, если в архиве его не было
if is_index_found is False:
  logs_folder = os.path.join('/content/CoverGen/logs', MODEL)
  index_path = os.path.join(logs_folder, 'model.index')
  if os.path.exists(index_path) == False:
    !wget 'https://huggingface.co/sail-rvc/2001MJAIDAM/resolve/main/model.index' -P {logs_folder}

!rm -r /content/unzips/
!rm -r /content/zips/

#@markdown ---

clear_output()

os.chdir('/content/CoverGen/assets/weights')
print("\nУстановленные модели:\n")
files = [f for f in os.listdir('.') if f.endswith('.pth')]
for file in files:
    print(file[:-4])


In [ ]:
#@title <big>🗣️ **Замена голоса**

import os
import ffmpeg
import IPython.display as ipd
from IPython.display import clear_output, display, Audio
from google.colab import files
from ipywidgets import Button

%cd /content/CoverGen

#@markdown ---

#@markdown **Введите имя модели:**
model_name = "Model_Name" #@param {type:"string"}

#@markdown ---

#@markdown **Метод обработки голоса:**
method = "rmvpe" #@param ["rmvpe", "pm", "harvest"] {allow-input: false}
#@markdown **Высота тона голоса ( -12 - на октаву ниже, 0 - сохраняет исходную высоту, 12 - на октаву выше ):**
pitch = 0 #@param {type:"slider", min:-12, max:12, step:1}

#@markdown ---

#@markdown **Дополнительные настройки:**

#@markdown <small> Контролирует, насколько акцент голоса ИИ будет сохранен в вокале:
index_rate = 0.66 #@param {type:"slider", min:0, max:1, step:0.01}
volume_normalization = 1
#@markdown <small> Защита безголосых согласных и звуков дыхания для предотвращения артефактов. Уменьшите значение, чтобы усилить защиту, но это может снизить точность индексирования:
consonant_protection = 0.2 #@param {type:"slider", min:0, max:0.5, step:0.01}
#@markdown <small> Если >=3: примените медианную фильтрацию к собранным результатам питча. Значение представляет собой радиус фильтрации и может уменьшить дыхание:
filter_radius = 3 #@param {type:"slider", min:0, max:7, step:1}
is_half = False
#@markdown **Формат выходного файла:**
format = "mp3" #@param ["mp3", "wav", "flac"] {allow-input: false}

input_path = '/content/output/ouvr/Music_vocals.mp3'
model_path = f'{model_name}.pth'
index_path = f'logs/{model_name}/added_*_{model_name}_v2.index'
opt_path = f'/content/output/no_processed/output.{format}'

# Удаление прошлого сгенерированного аудиофайла перед генерацией нового
!rm -r /content/output/no_processed/*

# Скрипт заменяющий голос
!python tools/infer_cli.py --f0up_key $pitch --input_path $input_path --index_path $index_path --f0method $method --opt_path $opt_path --model_name $model_path --index_rate $index_rate --device cuda --is_half $is_half --filter_radius $filter_radius --resample_sr 0 --rms_mix_rate $volume_normalization --protect $consonant_protection

#@markdown ---

output_path = '/content/output/no_processed'

result_file = os.path.join(output_path, input_name + "." + format)

!ffmpeg -y -i {opt_path} -i {instrumental} -filter_complex "[0:a][1:a]amerge=inputs=2[a]" -map "[a]" -ac 2 {result_file}

#@markdown <big><big> **Поставьте галочку если вылезла ошибка или не появился файл, после этого скиньте скрин ошибки в тг https://t.me/+GMTP7hZqY0E4OGRi**
check_error = False #@param {type:"boolean"}
if not check_error:
    ipd.clear_output()

# Вывод сгенерированного аудиофайла и создание кнопки "Скачать"
def download_file(file_path):
    files.download(file_path)

print("\nGoogle Colab иногда не может вывести окно с аудио, поэтому:\n")
print("\n1. Сначала выводим кнопку 'Скачать', после нажатия на которую вы можете сохранить аудиозапись на ваше устройство.\n")

download_button = Button(description="Скачать")
display(download_button)
download_button.on_click(lambda _: download_file(result_file))

print("\n2. Затем, Выводим саму аудиозапись для прослушивания прямо в Google Colab.\n")

audio = Audio(result_file, autoplay=False)
display(audio)

print("\nЭтот ai cover не содержит в себе никакой обработки. (ячейка с обработкой кавера будет позже)\n")


In [ ]:
#@title 💾  **Скачивание файлов**

#@markdown ---
#@markdown #<big> UVR

Download_UVR = "None" # @param ["Вокал", "Инструментал", "Вокал & Инструментал"] {allow-input: true}

if Download_UVR=="Вокал":
  print('Скачиваем Вокал...')
  files.download(f'/content/output/ouvr/Music_vocals.{format}')
elif Download_UVR=="Инструментал":
  print('Скачиваем Инструментал...')
  files.download(f'/content/output/ouvr/Music_instrum.{format}')
elif Download_UVR=="Вокал & Инструментал":
  print('Скачиваем Вокал и Инструментал...')
  files.download(f'/content/output/ouvr/Music_vocals.{format}')
  files.download(f'/content/output/ouvr/Music_instrum.{format}')

#@markdown ---
#@markdown #<big> Ai Cover

Download_Ai_Cover = "None" # @param ["Обработанный Вокал (без эффектов)", "Готовый Ai Cover (без эффектов)"] {allow-input: true}

if Download_Ai_Cover=="Обработанный Вокал (без эффектов)":
  print('Скачиваем Обработанный Вокал...')
  files.download(f'/content/output/no_processed/output.{format}')
elif Download_Ai_Cover=="Готовый Ai Cover (без эффектов)":
  print('Скачиваем Ai Cover...')
  files.download(f'/content/output/no_processed/Music.{format}')

#@markdown ---

# <font color='#FF8C00'> **<big> <<< UVR**

In [ ]:
#@title <big>🚀 **УСТАНОВКА**

from IPython.display import clear_output
from ipywidgets import Button

!mkdir -p /content/input

!git clone https://github.com/Bebra777228/UVR-Colab /content/UVR &> /dev/null
%cd /content/UVR

!pip install --no-cache-dir -qq -r requirements.txt &> /dev/null
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/  &> /dev/null

!mkdir -p /content/output
!rm -r /content/sample_data/

#@markdown **Подключить гугл диск *(более быстрая работа с файлами)***
GDrive = False #@param {type:"boolean"}
if GDrive:
  from google.colab import drive
  drive.mount('/content/drive')

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big>✂️ **РАЗДЕЛЕНИЕ**
from IPython.display import clear_output
from ipywidgets import Button
from pathlib import Path
import glob

%cd /content/UVR

#@markdown ---
#@markdown * **Путь к файлу, либо к папке с файлами:**
input = '/content/drive/MyDrive/input' #@param {type:"string"}
output = '/content/drive/MyDrive/output' #@param {type:"string"}
#@markdown > <small> В input папку можно загрузить несколько файлов.
#@markdown ---
#@markdown * **Качество разделения:**
quality = 3 #@param {type:"slider", min:1, max:10, step:1}
#@markdown ---
#@markdown * **Модели разделения:**
MDX23C_8KFFT_InstVoc_HQ = True # @param {type:"boolean"}
BS_Roformer = True # @param {type:"boolean"}
Mel_Band_Roformer = False # @param {type:"boolean"}
Vit_Large = False # @param {type:"boolean"}
UVR_MDX_NET_Inst_HQ_4 = False # @param {type:"boolean"}
UVR_MDX_NET_Voc_FT = False # @param {type:"boolean"}
#@markdown ---

use_InstVoc = '--use_InstVoc' if MDX23C_8KFFT_InstVoc_HQ is True else ''
use_BSRoformer = '--use_BSRoformer' if BS_Roformer is True else ''
use_MelRoformer = '--use_MelRoformer' if Mel_Band_Roformer is True else ''
use_VitLarge = '--use_VitLarge' if Vit_Large is True else ''
use_InstHQ4 = '--use_InstHQ4' if UVR_MDX_NET_Inst_HQ_4 is True else ''
use_VOCFT = '--use_VOCFT' if UVR_MDX_NET_Voc_FT is True else ''


if Path(input).is_file():
  file_path = input
  Path(output).mkdir(parents=True, exist_ok=True)
  !python inference.py \
        --input_audio "{file_path}" \
        --large_gpu \
        --weight_MelRoformer {quality} \
        --weight_BSRoformer {quality} \
        --weight_InstVoc {quality} \
        --weight_InstHQ4 {quality} \
        --weight_VOCFT {quality} \
        --weight_VitLarge {quality} \
        --overlap_MelRoformer {quality} \
        --overlap_BSRoformer {quality} \
        --overlap_demucs 1 \
        --overlap_VOCFT 1 \
        --overlap_InstHQ4 1 \
        --output_format "FLAC" \
        --BigShifts {quality} \
        --output_folder "{output}" \
        --vocals_only \
        {use_VitLarge} \
        {use_VOCFT} \
        {use_InstHQ4} \
        {use_MelRoformer} \
        {use_BSRoformer} \
        {use_InstVoc}


else:
    file_paths = sorted(glob.glob(input + "/*"))[:]
    input_audio_args = ' '.join([f'"{path}"' for path in file_paths])
    Path(output).mkdir(parents=True, exist_ok=True)
    !python inference.py \
        --input_audio {input_audio_args} \
        --large_gpu \
        --weight_MelRoformer {quality} \
        --weight_BSRoformer {quality} \
        --weight_InstVoc {quality} \
        --weight_InstHQ4 {quality} \
        --weight_VOCFT {quality} \
        --weight_VitLarge {quality} \
        --overlap_MelRoformer {quality} \
        --overlap_BSRoformer {quality} \
        --overlap_demucs 1 \
        --overlap_VOCFT 1 \
        --overlap_InstHQ4 1 \
        --output_format "FLAC" \
        --BigShifts {quality} \
        --output_folder "{output}" \
        --vocals_only \
        {use_VitLarge} \
        {use_VOCFT} \
        {use_InstHQ4} \
        {use_MelRoformer} \
        {use_BSRoformer} \
        {use_InstVoc}


# <font color='#FF8C00'> **<big> <<< GRАDIО**

## **<big><<< EasyGUI** <small><small>(Оригинал, на английском языке)

In [ ]:
#@title <font color='#00FFFF'><big> **Установить RVC** ⬇️
%cd /content
from IPython.display import clear_output
from ipywidgets import Button
import subprocess, shlex, os
from google.colab import drive

var = "We"+"bU"+"I"
test = "Voice"
c_word = "Conversion"
r_word = "Retrieval"
!git clone https://github.com/RVC-Project/{r_word}-based-{test}-{c_word}-{var} /content/RVC

!apt -y install -qq aria2
pretrains = ["f0D32k.pth","f0G32k.pth"]
new_pretrains = ["f0Ov2Super32kD.pth","f0Ov2Super32kG.pth"]

for file in pretrains:
    if not os.path.exists(f"/content/RVC/assets/pretrained_v2/{file}"):
        command = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/%s%s%s/resolve/main/pretrained_v2/%s -d /content/RVC/assets/pretrained_v2 -o %s" % ("Voice","Conversion","WebUI",file,file)
        try:
            subprocess.run(shlex.split(command))
        except Exception as e:
            print(e)

for file in new_pretrains:
    if not os.path.exists(f"/content/RVC/assets/pretrained_v2/{file}"):
        command = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/poiqazwsx/Ov2Super32kfix/resolve/main/%s -d /content/RVC/assets/pretrained_v2 -o %s" % (file,file)
        try:
            subprocess.run(shlex.split(command))
            print(shlex.split(command))
        except Exception as e:
            print(e)

!mkdir -p /content/dataset && mkdir -p /content/RVC/audios
!wget -nc https://raw.githubusercontent.com/RejektsAI/EasyTools/main/original -O /content/RVC/original.py
!wget -nc https://raw.githubusercontent.com/RejektsAI/EasyTools/main/app.py -O /content/RVC/demo.py
!wget -nc https://raw.githubusercontent.com/RejektsAI/EasyTools/main/easyfuncs.py -O /content/RVC/easyfuncs.py
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/download_files.py -O /content/RVC/download_files.py
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/a.png -O /content/RVC/a.png
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/easy_sync.py -O /content/RVC/easy_sync.py
!wget -nc https://huggingface.co/spaces/Rejekts/RVC_PlayGround/raw/main/app.py -O /content/RVC/playground.py
!wget -nc https://huggingface.co/spaces/Rejekts/RVC_PlayGround/raw/main/tools/useftools.py -O /content/RVC/tools/useftools.py
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/astronauts.mp3 -O /content/RVC/audios/astronauts.mp3
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/somegirl.mp3 -O /content/RVC/audios/somegirl.mp3
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/someguy.mp3 -O /content/RVC/audios/someguy.mp3
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/unchico.mp3 -O /content/RVC/audios/unchico.mp3
!wget -nc https://huggingface.co/Rejekts/project/resolve/main/unachica.mp3 -O /content/RVC/audios/unachica.mp3
!cd /content/RVC && python /content/RVC/download_files.py

if not "installed" in locals():
    !cd /content/RVC && pip install -r requirements.txt
    !pip install mega.py gdown==4.6.0 pytube pydub  gradio==3.42.0
installed=True

save_to_drive=False#@param {type:"boolean"}

if save_to_drive:
    try:
        from google.colab import auth
        from pydrive2.auth import GoogleAuth
        from oauth2client.client import GoogleCredentials
        from pydrive2.drive import GoogleDrive
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        my_drive = GoogleDrive(gauth)
        drive.mount('/content/drive')
        drive_trash = my_drive.ListFile({'q': "trashed = true"}).GetList()
        from RVC.easy_sync import GarbageMan
        kevin = GarbageMan()
        kevin.start(path=drive_trash,every=40,pattern="[GD]_*.pth")
    except Exception as e:
        print(e)
from RVC.easy_sync import Channel
logs_folder ='/content/drive/MyDrive/project-main/logs'
weights_folder = '/content/drive/MyDrive/project-main/assets/weights'
if not "logs_backup" in locals(): logs_backup = Channel('/content/RVC/logs',logs_folder,every=30,exclude="mute")
if not "weights_backup" in locals(): weights_backup = Channel('/content/RVC/assets/weights',weights_folder,every=30)

if os.path.exists('/content/drive/MyDrive'):
    if not os.path.exists(logs_folder): os.makedirs(logs_folder)
    if not os.path.exists(weights_folder): os.makedirs(weights_folder)
    logs_backup.start()
    weights_backup.start()

clear_output()
Button(description="\u2714 Success", button_style="success")

In [ ]:
#@title <font color='#00FFFF'><big> **Открыть интерфейс** 📱

#@markdown ---

import shutil
%cd /content/RVC
load_models_from_drive = False #@param{type:"boolean"}
open_tensorboard = True #@param{type:"boolean"}
py = "demo.py"
if load_models_from_drive:
    if os.path.exists('/content/drive/MyDrive/project-main'):
        for file in os.listdir('/content/drive/MyDrive/project-main/assets/weights'):
            try: shutil.copy2(f'/content/drive/MyDrive/project-main/assets/weights/{file}','/content/RVC/assets/weights/')
            except: print(f"Error loading {file}")
        for file in os.listdir('/content/drive/MyDrive/project-main/logs'):
            try: shutil.copytree(f'/content/drive/MyDrive/project-main/logs/{file}',f'/content/RVC/logs/{file}')
            except: print(f"Error loading {file}")
    else:
        print("Google Drive not connected...")
if open_tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./logs --port=8888
!python {py} --colab

#@markdown ---

# <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>

**[<center><big><big><big><big><big> Оригинальный блокнот </center>](https://colab.research.google.com/drive/1r4IRL0UA7JEoZ0ZK8PKfMyTIBHKpyhcw?usp=sharing)**
# <small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.</small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small></small>